In [2]:
import random
import time

In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [4]:
import tensorflow as tf
import numpy as np

np.set_printoptions(precision=4, suppress=True)

In [5]:
import tensorflow.contrib.layers

In [6]:
import gym

In [18]:
env = gym.make('CartPole-v1')
# env = gym.make('LunarLander-v2')

[2017-01-25 19:44:11,510] Making new env: CartPole-v1


In [19]:
env.observation_space

Box(4,)

In [8]:
def CartPoleAC(state, is_training):
    hidden = tf.contrib.layers.fully_connected(
        state, 32,
        activation_fn=tf.nn.elu,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
        scope='hidden1')
    hidden = tf.contrib.layers.fully_connected(
        hidden, 32,
        activation_fn=tf.nn.elu,
        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
        scope='hidden2')

    baseline = tf.contrib.layers.linear(hidden, 1,
                                     weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                                     biases_initializer=tf.constant_initializer(0.),
                                     scope='baseline')
    logits = tf.contrib.layers.linear(hidden, env.action_space.n,  # num_actions
                                      weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                      scope='logits')
    return logits, baseline

In [9]:
ROLLOUT_LEN = 20
GAMMA = 0.99

In [10]:
import learner
reload(learner)

<module 'learner' from 'learner.pyc'>

In [20]:
sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [21]:
with tf.device('/cpu:0'):
    ac = learner.ActorCriticLearner(CartPoleAC, env.observation_space.shape[0],
                                    options={'clip_grad': 9.,
                                             'learning_rate': 0.0001,
                                             'update_steps': 15000
                                            })

INFO:tensorflow:Summary name model/hidden1/weights:0/grad is illegal; using model/hidden1/weights_0/grad instead.


[2017-01-25 19:44:15,801] Summary name model/hidden1/weights:0/grad is illegal; using model/hidden1/weights_0/grad instead.


INFO:tensorflow:Summary name model/hidden1/weights:0 is illegal; using model/hidden1/weights_0 instead.


[2017-01-25 19:44:15,804] Summary name model/hidden1/weights:0 is illegal; using model/hidden1/weights_0 instead.


INFO:tensorflow:Summary name model/hidden1/biases:0/grad is illegal; using model/hidden1/biases_0/grad instead.


...

[2017-01-25 19:44:15,859] Summary name model/logits/biases:0/grad is illegal; using model/logits/biases_0/grad instead.


INFO:tensorflow:Summary name model/logits/biases:0 is illegal; using model/logits/biases_0 instead.


[2017-01-25 19:44:15,863] Summary name model/logits/biases:0 is illegal; using model/logits/biases_0 instead.


INFO:tensorflow:Summary name global_step:0 is illegal; using global_step_0 instead.


[2017-01-25 19:44:15,868] Summary name global_step:0 is illegal; using global_step_0 instead.


In [22]:
ac.Init(sess, 'practice/cartpole-ac1')

In [ ]:
# %%prun
episode_rew = 0.
episode_len = 0.
old_s = env.reset()
for i in range(10000000):
    ss, aa, rr, ss1, gg = [], [], [], [], []
    done = False
    while not done and len(ss) < ROLLOUT_LEN:
        a = ac.SampleAction(sess, old_s.reshape([1, -1]))
        s, r, done, _ = env.step(a)

        ss.append(old_s)
        aa.append(a)
        rr.append(r)
        ss1.append(s)
        gg.append(GAMMA if not done else 0.0)

        episode_rew += r
        episode_len += 1

        old_s = s

    rew = 0.
    g = 1.
    for i in reversed(range(len(ss))):
        rew = rr[i] + gg[i] * rew
        g *= gg[i]
        ss1[i] = old_s
        rr[i] = r
        gg[i] = g

    if done:
        ac.Stat({'Env/Reward': episode_rew, 'Env/Length': episode_len})
        episode_rew, episode_len = 0., 0.
        old_s = env.reset()

    ac.Step(sess, (ss, aa, rr, old_s.reshape([1, -1]), gg))

In [17]:
ss

[array([[ 0.012 ,  0.0156,  0.0232,  0.0319]]),
 array([[ 0.0123, -0.1799,  0.0238,  0.3318]]),
 array([[ 0.0087, -0.3753,  0.0305,  0.6319]]),
 array([[ 0.0012, -0.5709,  0.0431,  0.9341]]),
 array([[-0.0102, -0.7665,  0.0618,  1.24  ]]),
 array([[-0.0255, -0.5723,  0.0866,  0.9673]]),
 array([[-0.037 , -0.3784,  0.1059,  0.703 ]]),
 array([[-0.0446, -0.5748,  0.12  ,  1.0271]]),
 array([[-0.0561, -0.3815,  0.1405,  0.7743]]),
 array([[-0.0637, -0.1885,  0.156 ,  0.529 ]]),
 array([[-0.0675,  0.0041,  0.1666,  0.2892]]),
 array([[-0.0674, -0.193 ,  0.1724,  0.6295]]),
 array([[-0.0712, -0.39  ,  0.185 ,  0.9711]]),
 array([[-0.079 , -0.5871,  0.2044,  1.3157]])]